In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json

In [5]:


def build_alexnet(input_shape, num_classes):
    """
    Builds a simplified version of AlexNet for classification.
    """
    model = Sequential([
        Conv2D(64, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Conv2D(192, (5, 5), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Conv2D(384, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def create_results_folder(model_name, augmentation_type):
    """Create a folder to store results."""
    results_folder = f"results_{model_name}_{augmentation_type}"
    os.makedirs(results_folder, exist_ok=True)
    return results_folder

def save_training_logs(results_folder, metrics):
    """Save training metrics to a JSON file."""
    with open(os.path.join(results_folder, "training_logs.json"), "w") as f:
        json.dump(metrics, f, indent=4)

def save_loss_curve(results_folder, history):
    """Save loss and accuracy curves."""
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(results_folder, "training_curves.png"))
    plt.close()

def save_model_checkpoint(results_folder, model):
    """Save the model weights."""
    model.save(os.path.join(results_folder, "final_model.keras"))

def plot_confusion_matrix(cm, class_names):
    """Plots and returns a confusion matrix heatmap using seaborn."""
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    return plt

def plot_roc_curve(y_true, y_pred_prob, num_classes):
    """Plots and returns the ROC curve for multi-class classification."""
    y_true_bin = label_binarize(y_true, classes=range(num_classes))
    fpr, tpr, roc_auc = {}, {}, {}
    plt.figure(figsize=(8, 8))

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    return fpr, tpr, roc_auc

def print_generator_info(train_generator):
    """Print information about the data generator."""
    print("Class names and their corresponding indices:")
    for class_name, class_index in train_generator.class_indices.items():
        print(f"Class: {class_name}, Index: {class_index}")

    print("\nClass distribution (number of images in each class):")
    for class_name, num_images in train_generator.class_indices.items():
        class_folder = os.path.join(train_generator.directory, class_name)
        image_count = len([f for f in os.listdir(class_folder) 
                         if os.path.isfile(os.path.join(class_folder, f))])
        print(f"Class: {class_name}, Number of images: {image_count}")

def load_data(train_path, val_path):
    """Load and preprocess the image data using ImageDataGenerator."""
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )

    train_generator = datagen.flow_from_directory(
        train_path,
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical',
        color_mode='grayscale'
    )

    test_generator = datagen.flow_from_directory(
        val_path,
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical',
        color_mode='grayscale'
    )

    return train_generator, test_generator

def main():
    # Configuration
    input_shape = (256, 256, 1)
    num_classes = 4
    model_name = "AlexNet"
    augmentation_type = "simple"

    # Create results folder
    results_folder = create_results_folder(model_name, augmentation_type)
    print(f"Results will be saved in: {results_folder}")

    # Load data
    train_generator, test_generator = load_data(
        r"C:\Users\Vinay Dagar\Downloads\Thyroid data\DU\balanced_output\train",
        r"C:\Users\Vinay Dagar\Downloads\Thyroid data\DU\balanced_output\val"
    )

    # Print data information
    print_generator_info(train_generator)

    # Callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        ModelCheckpoint(
            filepath=os.path.join(results_folder, 'best_model.keras'),
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            verbose=1
        )
    ]

    # Build and train model
    model = build_alexnet(input_shape, num_classes)
    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=15,
        batch_size=16,
        callbacks=callbacks
    )

    # Save results
    metrics_to_log = {
        "accuracy": history.history['accuracy'][-1],
        "val_accuracy": history.history['val_accuracy'][-1],
        "loss": history.history['loss'][-1],
        "val_loss": history.history['val_loss'][-1]
    }
    save_training_logs(results_folder, metrics_to_log)
    save_loss_curve(results_folder, history)
    save_model_checkpoint(results_folder, model)

    # Generate and save evaluation plots
    y_true = test_generator.classes
    y_pred_prob = model.predict(test_generator)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_plot = plot_confusion_matrix(cm, list(train_generator.class_indices.keys()))
    cm_plot.savefig(os.path.join(results_folder, "confusion_matrix.png"))
    plt.close()

    # Classification Report
    class_report = classification_report(
        y_true, 
        y_pred, 
        target_names=list(train_generator.class_indices.keys()),
        output_dict=True
    )
    with open(os.path.join(results_folder, "classification_report.json"), "w") as f:
        json.dump(class_report, f, indent=4)

    # ROC Curve
    fpr, tpr, roc_auc = plot_roc_curve(y_true, y_pred_prob, num_classes)
    plt.savefig(os.path.join(results_folder, "roc_curve.png"))
    plt.close()

    print(f"Results saved in: {results_folder}")

if __name__ == "__main__":
    main()

Results will be saved in: results_AlexNet_simple
Found 290 images belonging to 4 classes.
Found 74 images belonging to 4 classes.
Class names and their corresponding indices:
Class: class_0, Index: 0
Class: class_1, Index: 1
Class: class_2, Index: 2
Class: class_3, Index: 3

Class distribution (number of images in each class):
Class: class_0, Number of images: 72
Class: class_1, Number of images: 70
Class: class_2, Number of images: 73
Class: class_3, Number of images: 75


C:\Users\Vinay Dagar\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/15


C:\Users\Vinay Dagar\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 604ms/step - accuracy: 0.2564 - loss: 1.5148
Epoch 1: val_loss improved from inf to 1.32381, saving model to results_AlexNet_simple\best_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 981ms/step - accuracy: 0.2566 - loss: 1.5095 - val_accuracy: 0.2973 - val_loss: 1.3238
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 558ms/step - accuracy: 0.4783 - loss: 1.1881
Epoch 2: val_loss improved from 1.32381 to 1.30087, saving model to results_AlexNet_simple\best_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 878ms/step - accuracy: 0.4821 - loss: 1.1868 - val_accuracy: 0.4459 - val_loss: 1.3009
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 604ms/step - accuracy: 0.4776 - loss: 1.1600
Epoch 3: val_loss improved from 1.30087 to 1.02896, saving model to results_AlexNet_simple\best_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 900ms/step - accuracy: 0.4812 - loss: 1.1497 - val_accuracy: 0.5270 - val_loss: 1.0290
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 586ms/step - accuracy: 0.5826 - loss: 0.